# Notebook for Autodiff
Experiments with automatic differentiation

In [1]:
# %% Import JAX and enable 64-bit precision
import jax
jax.config.update("jax_enable_x64", True)

In [2]:
# %% Import other libraries
import equinox as eqx
import interpax
import diffrax
import optimistix as optx
import optax

import jax.numpy as jnp
import jax.tree_util as jtu
from dataclasses import dataclass
from functools import partial
import numpy as np
from collections import namedtuple
from typing import NamedTuple
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyvista as pv

from netCDF4 import Dataset

from jaxtyping import ArrayLike, Real

In [ ]:
# %% Load my own libraries
%load_ext autoreload
%autoreload 2
from c1lgkt.jax.fields.equilibrium import Equilibrium
from c1lgkt.jax.fields.clebsch import ThetaMapping, ClebschMappingBuilder
from c1lgkt.jax.fields.field_providers import AbstractFieldProvider, ZonalFieldProvider
from c1lgkt.jax.particles.particle_motion import ParticleParams

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# %% Load equilibrium
eq = Equilibrium.from_eqdfile('./tests/D3D141451.eqd')

In [54]:
fields = [ZonalFieldProvider(interpax.Interpolator1D(jnp.linspace(0, 1, 10), jnp.zeros(10)), interpax.Interpolator1D(jnp.linspace(0, 1, 10), jnp.zeros(10)+1.0)),
          ZonalFieldProvider(interpax.Interpolator1D(jnp.linspace(0, 1, 10), jnp.zeros(10)+1.0), interpax.Interpolator1D(jnp.linspace(0, 1, 10), jnp.zeros(10))),
          ZonalFieldProvider(interpax.Interpolator1D(jnp.linspace(0, 1, 10), jnp.zeros(10)+1.0), interpax.Interpolator1D(jnp.linspace(0, 1, 10), jnp.zeros(10))),]

[ZonalFieldProvider(
   interp_phi=Interpolator1D(
     method='cubic',
     x=f64[10],
     f=f64[10],
     derivs={'fx': f64[10]},
     extrap=False,
     period=None,
     axis=0
   ),
   interp_ap=Interpolator1D(
     method='cubic',
     x=f64[10],
     f=f64[10],
     derivs={'fx': f64[10]},
     extrap=False,
     period=None,
     axis=0
   )
 )]

In [24]:
fields[0].value_and_grad(0.2, jnp.linspace(0.1, 0.3, 10), jnp.linspace(0, 2*jnp.pi, 10), jnp.zeros(10))

((Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
  Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
 ((Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
   Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
  (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
   Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
  (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
   Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64))))

In [41]:
psi = jnp.linspace(0.1, 0.3, 10)
theta = jnp.linspace(0, 2*jnp.pi, 10)
varphi = jnp.zeros_like(psi)

cum_fields = fields[0].value_and_grad(0.2, psi, theta, varphi)

for f in fields:
    val_and_grad = f.value_and_grad(0.2, psi, theta, varphi)
    cum_fields = jax.lax.add(cum_fields, val_and_grad)

TypeError: Argument '((JitTracer<float64[10]>, JitTracer<float64[10]>), ((JitTracer<float64[10]>, JitTracer<float64[10]>), (JitTracer<float64[10]>, JitTracer<float64[10]>), (JitTracer<float64[10]>, JitTracer<float64[10]>)))' of type '<class 'tuple'>' is not a valid JAX type

In [46]:
sum(vals)

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [47]:
vals

[((Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
   Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64)),
  ((Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)))),
 ((Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64),
   Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
  ((Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 

In [48]:
jax.lax.add(vals[0], vals[1])

TypeError: Argument '((JitTracer<float64[10]>, JitTracer<float64[10]>), ((JitTracer<float64[10]>, JitTracer<float64[10]>), (JitTracer<float64[10]>, JitTracer<float64[10]>), (JitTracer<float64[10]>, JitTracer<float64[10]>)))' of type '<class 'tuple'>' is not a valid JAX type

In [56]:
from functools import reduce
vals = [f.value_and_grad(0.2, psi, theta, varphi) for f in fields]
reduce(lambda a, b: jax.tree.map(lambda x, y: x + y, a, b), vals)

((Array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float64),
  Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64)),
 ((Array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float64),
   Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64)),
  (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
   Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
  (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
   Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64))))

In [50]:
vals

[((Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
   Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64)),
  ((Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)))),
 ((Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64),
   Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
  ((Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64),
    Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float64)),
   (Array([0., 0., 0., 0., 0., 0., 0., 0., 